<a href="https://colab.research.google.com/github/ichekhovskikh/elibrary-pdf-downloader/blob/master/elibrary_pdf_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install requests
!pip install tika

import requests
import re
from io import BytesIO
from tika import parser
from google.colab import drive

Указание логина и пароля для доступа к eLibrary. так же требуется указать строку поиска и количество страниц для просмотра

In [0]:
login = 'ivanivanivan2'
password = 'ivan1234'
search_tags = ['агро производство', 'анализ изображений', 'космические снимки']
page_count = 5

Монтирование директории для работы с файлами

In [0]:
drive.mount('/content/drive')

Инициализация сессии для работы с eLibrary

In [0]:
save_path = 'data.json'

headers = {
    'content-type': 'application/x-www-form-urlencoded',
    'user-agent': 'PostmanRuntime/7.19.0'
}
params = {'login': login, 'password': password}

auth_url = 'https://elibrary.ru/start_session.asp'
search_url = 'https://elibrary.ru/query_results.asp?where_abstract=on&type_article=on&queryboxid=&type_report=on&type_preprint=on&search_open=&rubrics_all=&changed=1&where_fulltext=on&where_keywords=on&type_patent=on&search_freetext=&search_fulltext=&where_name=on&where_references=on&itemboxid=&end_year=&titles_all=&where_affiliation=on&begin_year=&order=rev&type_conf=on&search_morph=on&authors_all=&type_disser=on&search_results=&type_book=on&ftext={}&issues=all&orderby=rank'
search_page_url = 'https://elibrary.ru/query_results.asp?pagenum={}'
download_pdf_url = 'https://elibrary.ru/full_text.asp?id={}'

session = requests.Session()
session.post(auth_url, data = params, headers = headers)

Получение идентификаторов статей

In [0]:
def get_all_article_ids(html_page):
   matches = re.findall(r"#pdf_\d+", html_page)
   for match in matches:
     yield match.replace("#pdf_", "")

Скачивание статьи

In [0]:
def download_pdf(pdf_id):
  response = session.get(download_pdf_url.format(pdf_id), headers=headers, stream=True)
  return BytesIO(response.content)

Конвертирование pdf в text

In [0]:
ignored_text_fragments = [
  'eLIBRARY RU НАУЧНАЯ ЭЛЕКТРОННАЯ БИБЛИОТЕКА', 
  'Содержание корзины ПОИСК Найти Расширенный поиск НАВИГАТОР',
  'Из за нарушения правил пользования сайтом eLIBRARY RU'
] 

def pdf2text(pdf_file):
  text = parser.from_buffer(pdf_file)['content']
  only_words = re.sub(r'\W+', r' ', text)
  for ignored_fragment in ignored_text_fragments:
    if (ignored_fragment in only_words):
      return None
  return only_words

Конвертирование text в json

In [0]:
def get_acticle_as_json(pdf_id, text):
  return '{"id":' + pdf_id + ',"text":"' + text + '","tags":["' + pdf_id + '"]}'

In [0]:
json_file = "["

for tag in search_tags:
  session.get(search_url.format(tag), headers=headers)
  for page_index in range(page_count):
    print('search = {}, page = {}'.format(tag, page_index + 1))
    response = session.get(search_page_url.format(page_index + 1), headers=headers)
    pdf_ids = list(get_all_article_ids(response.text))
    print('pdf_ids = {}'.format(pdf_ids))
    if len(pdf_ids) < 1:
      break

    for pdf_id in pdf_ids:
      try:
        pdf_file = download_pdf(pdf_id)
        text = pdf2text(pdf_file)
        if (text is None):
          continue
        print('loaded {}'.format(pdf_id))
        acticle_json = get_acticle_as_json(pdf_id, text)
        json_file += acticle_json + ","
      except:
        continue

if len(json_file) > 1:
  json_file = json_file[:-1]
json_file += "]"
with open(save_path, 'w') as save_file:
  save_file.write(json_file)